# Settings

In [ ]:

import os
import sys
import subprocess
import wave
import aifc
import math
import audioop
import collections
import json
import base64
import threading
import platform
import stat
import hashlib
import hmac
import time
import uuid

In [1]:
import io
import pyaudio

# Source

In [40]:
audio = pyaudio.PyAudio()

In [41]:
count = audio.get_device_count()

In [42]:
audio.get_device_info_by_index(0)

{'index': 0,
 'structVersion': 2,
 'name': 'Built-in Microphone',
 'hostApi': 0,
 'maxInputChannels': 2,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.0029478458049886623,
 'defaultLowOutputLatency': 0.01,
 'defaultHighInputLatency': 0.01310657596371882,
 'defaultHighOutputLatency': 0.1,
 'defaultSampleRate': 44100.0}

In [43]:
device_info = audio.get_device_info_by_index(0)
sample_rate = int(device_info["defaultSampleRate"])

In [44]:
#audio.terminate()

In [45]:
device_index = 0

In [46]:
form = pyaudio.paInt16

In [47]:
SAMPLE_WIDTH = pyaudio.get_sample_size(form)

In [48]:
SAMPLE_RATE = sample_rate

In [49]:
CHUNK = 1024

In [50]:
#audio = None
stream = None

In [60]:
class MicrophoneStream(object):
    def __init__(self, pyaudio_stream):
        self.pyaudio_stream = pyaudio_stream

    def read(self, size):
        return self.pyaudio_stream.read(size, exception_on_overflow=False)

    def close(self):
        try:
            # sometimes, if the stream isn't stopped, closing the stream throws an exception
            if not self.pyaudio_stream.is_stopped():
                self.pyaudio_stream.stop_stream()
        finally:
            self.pyaudio_stream.close()

In [65]:
stream = MicrophoneStream(
    audio.open(
        input_device_index=device_index, channels=1, format=form,
        rate=SAMPLE_RATE, frames_per_buffer=CHUNK, input=True,
    )
)

In [71]:
stream.close()

# Recogniser

In [66]:
energy_threshold = 300
dynamic_energy_treshold = True
dynamic_energy_adjustment_damping = 0.15
dynamic_energy_ratio = 1.5
pause_threshold = 0.8  # seconds of non-speaking audio before a phrase is considered complete
operation_timeout = None  # seconds after an internal operation (e.g., an API request) starts before it times out, or ``None`` for no timeout

phrase_threshold = 0.3  # minimum seconds of speaking audio before we consider the speaking audio a phrase - values below this are ignored (for filtering out clicks and pops)
non_speaking_duration = 0.5  # seconds of non-speaking audio to keep on both sides of the recording

In [67]:
def record(duration=None, offset=None):
    frames = io.BytesIO()
    seconds_per_buffer = (CHUNK + 0.0) / SAMPLE_RATE
    elapsed_time = 0
    offset_time = 0
    offset_reached = False
    while True:  # loop for the total number of chunks needed
        if offset and not offset_reached:
            offset_time += seconds_per_buffer
            if offset_time > offset:
                offset_reached = True

        buffer = stream.read(CHUNK)
        if len(buffer) == 0: break
    
        print(elapsed_time)
        if offset_reached or not offset:
            elapsed_time += seconds_per_buffer
            if duration and elapsed_time > duration: 
                break

            frames.write(buffer)

    frame_data = frames.getvalue()
    frames.close()
    return frame_data
    #return AudioData(frame_data, SAMPLE_RATE, SAMPLE_WIDTH)

In [70]:
bla = record(0.5)

0
0.023219954648526078
0.046439909297052155
0.06965986394557823
0.09287981859410431
0.11609977324263039
0.13931972789115646
0.16253968253968254
0.18575963718820862
0.2089795918367347
0.23219954648526078
0.25541950113378686
0.2786394557823129
0.30185941043083897
0.325079365079365
0.3482993197278911
0.37151927437641713
0.3947392290249432
0.41795918367346924
0.4411791383219953
0.46439909297052134
0.4876190476190474


In [74]:
bla

b'\x05\x00\x05\x00\x03\x00\x05\x00\x03\x00\x00\x00\xff\xff\xfd\xff\xfe\xff\xfd\xff\xfd\xff\xff\xff\xff\xff\xff\xff\xfe\xff\xff\xff\x00\x00\xfd\xff\xfd\xff\xfd\xff\xfb\xff\xfd\xff\xfd\xff\xfc\xff\xfe\xff\x00\x00\x00\x00\xff\xff\x00\x00\x00\x00\x02\x00\x00\x00\xfc\xff\xfe\xff\x00\x00\x02\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xfd\xff\xff\xff\xff\xff\x00\x00\xff\xff\x00\x00\x01\x00\x01\x00\x00\x00\x01\x00\x03\x00\x01\x00\x00\x00\x01\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\x01\x00\x03\x00\x01\x00\x02\x00\x00\x00\xff\xff\x00\x00\x01\x00\x00\x00\x00\x00\x02\x00\x05\x00\x04\x00\x04\x00\x02\x00\x01\x00\x00\x00\x01\x00\x04\x00\x05\x00\x07\x00\x05\x00\x02\x00\x03\x00\x04\x00\x05\x00\x03\x00\x05\x00\x05\x00\x07\x00\n\x00\t\x00\t\x00\x07\x00\x08\x00\x08\x00\x07\x00\x08\x00\x08\x00\x05\x00\x05\x00\x07\x00\x05\x00\x07\x00\x07\x00\x06\x00\x05\x00\x05\x00\x05\x00\x03\x00\x02\x00\x03\x00\x01\x00\x01\x00\x03\x00\x04\x00\x04\x00\x05\x00\x05\x00\x03